<a href="https://colab.research.google.com/github/amanzoni1/fine_tuning/blob/main/Qwen3_RL_GRPO_Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Qwen3 Reinforcement Learning & GRPO with Reasoning

In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
from datasets import load_dataset
import numpy as np, transformers
tok = transformers.AutoTokenizer.from_pretrained("Qwen/Qwen3-1.7B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/9.73k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [3]:
ds = load_dataset(
        "dennlinger/eur-lex-sum",
        "english",
        trust_remote_code=True,
        split="train[:500]")

README.md:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

eur-lex-sum.py:   0%|          | 0.00/4.45k [00:00<?, ?B/s]

train.json:   0%|          | 0.00/93.9M [00:00<?, ?B/s]

validation.json:   0%|          | 0.00/24.1M [00:00<?, ?B/s]

test.json:   0%|          | 0.00/24.4M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [4]:
print(ds.column_names)

['celex_id', 'reference', 'summary']


In [6]:
from pprint import pprint
pprint(ds[0])

Output streaming troncato alle ultime 5000 righe.
              'thereof must be stated.\n'
              '√\n'
              '√\n'
              'Item 1.16\n'
              'Where applicable, the potential impact on the investment in the '
              'event of resolution under the Directive 2014/59/EU.\n'
              '√\n'
              'Item 1.17\n'
              'Lock-up agreements\n'
              '(a)\n'
              'The parties involved.\n'
              '(b)\n'
              'Content and exceptions of the agreement.\n'
              '(c)\n'
              'An indication of the period of the lock up.\n'
              '√\n'
              '√\n'
              'Item 1.18\n'
              'Information about selling shareholders if any.\n'
              '√\n'
              '√\n'
              'Item 1.18.1\n'
              'Name and business address of the person or entity offering to '
              'sell the underlying shares, the nature of any position office '
              'o

In [7]:
len_cel = ds.map(lambda eg: {"tok_len": len(tok(eg["celex_id"]).input_ids)})
len_ref = ds.map(lambda eg: {"tok_len": len(tok(eg["reference"]).input_ids)})
len_sum = ds.map(lambda eg: {"tok_len": len(tok(eg["summary"]).input_ids)})

print(np.percentile(len_cel["tok_len"], [50, 90, 99]))
print(np.percentile(len_ref["tok_len"], [50, 90, 99]))
print(np.percentile(len_sum["tok_len"], [50, 90, 99]))

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (218223 > 131072). Running this sequence through the model will result in indexing errors


Map:   0%|          | 0/500 [00:00<?, ? examples/s]

[10. 14. 14.]
[  8964.5   39795.8  219961.72]
[1059.   1904.1  3796.19]
